In [60]:
import numpy as np
from scipy import ndimage
import glob, os
import pandas as pd

In [61]:
def corners(image, h_min, w_min):

    y_max = len(image)
    x_max = len(image[0])

    y0, x0 = 0, 0 
    s = image[y0:y0+h_min, x0:x0+w_min].sum() 
    while s>0 and x0<x_max//2:
        x0 =x0+1
        y0 = 0
        while s>0 and y0<y_max//2:
            y0 = y0+1
            s = image[y0:y0+h_min, x0:x0+w_min].sum() 
            
    y1 = y_max-1 
    x1 = x_max-1
    s = image[y1-h_min:y1, x1-w_min:x1].sum() 
    while s>0 and x1>x_max//2:
        x1 =x1-1
        y1 = y_max
        while s>0 and y1>y_max//2:
            y1 = y1-1
            s = image[y1-h_min:y1, x1-w_min:x1].sum() 
            
    return y0, y1, x0, x1

In [62]:
folder_sim = "/home/fdutka/Dropbox/ProjektyNaukowe/Dissolution/OpenFoam/Disk_2018_1"
#folder_sim = os.getcwd()
files_list = glob.glob(os.path.join(folder_sim, "S*.png"))
files_list = np.sort(np.array(files_list))

In [63]:
tsh  = 120

f1 = files_list[0]
image = ndimage.imread(f1, flatten = True)

image2 = (image<tsh).astype(int)
y0, y1, x0, x1 = corners(image2, 200, 200)

areas = []
for f1 in files_list:
    image = ndimage.imread(f1, flatten = True)
    image2 = (image<tsh).astype(int)
    image2[:y0] = 0
    image2[y1:] = 0
    image2[:,:x0]=0
    image2[:,x1:]=0

    areas.append([f1.split("/")[-1], image2.sum()])


In [65]:
areas = np.array(areas)

In [81]:
area0 = 100*np.pi
a1 = areas[:,1].astype(int)
a1 = area0*a1/a1[0]

df1 = pd.DataFrame(np.vstack((areas[:,1],a1)).T, index = areas[:,0], columns = ["area_px2","area_mm2"])
df1.to_csv(os.path.join(folder_sim, "S_areas.csv"))